In [36]:
!apt-get update
!apt-get install -y chromium-browser
!apt install chromium-chromedriver

Reading package lists... Done
E: Could not open lock file /var/lib/apt/lists/lock - open (13: Permission denied)
E: Unable to lock directory /var/lib/apt/lists/
W: Problem unlinking the file /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permission denied)
W: Problem unlinking the file /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [37]:
from selenium import webdriver
import pandas as pd

In [38]:
!ls /usr/lib/chromium-browser/chromedriver

/usr/lib/chromium-browser/chromedriver


In [39]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [40]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select


In [41]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.headless = True
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://store.steampowered.com/charts/mostplayed")

In [42]:
from selenium.webdriver.common.action_chains import ActionChains


In [43]:
class_name_of_rank_game = "weeklytopsellers_RankCell_34h48"
class_name_of_link_game = "weeklytopsellers_TopChartItem_2C5PJ"
class_name_of_name_game = "weeklytopsellers_GameName_1n_4-"
class_name_of_price_game = "weeklytopsellers_PriceCell_3IyfU"
class_name_of_current_player = "weeklytopsellers_ConcurrentCell_3L0CD"
class_name_of_peak_player = "weeklytopsellers_PeakInGameCell_yJB7D"

In [44]:
# rank_game_elements = driver.find_elements(By.CLASS_NAME, class_name_of_rank_game)
rank_games=[]
while len(rank_games)==0:
    rank_games = [element.text for element in driver.find_elements(By.CLASS_NAME, class_name_of_rank_game)]
    name_games = [element.text for element in driver.find_elements(By.CLASS_NAME, class_name_of_name_game)]
    price_games = [element.text for element in driver.find_elements(By.CLASS_NAME, class_name_of_price_game)]
    current_players = [element.text for element in driver.find_elements(By.CLASS_NAME, class_name_of_current_player)]
    peak_players = [element.text for element in driver.find_elements(By.CLASS_NAME, class_name_of_peak_player)]
    link_games = [element.get_attribute("href") for element in  driver.find_elements(By.CLASS_NAME, class_name_of_link_game)]

In [45]:
data = {"rank": rank_games,
        "name": name_games,
        "price": price_games,
        "current_player": current_players,
        "peak_player": peak_players,
        "link": link_games
        }
df = pd.DataFrame(data)

In [46]:
# Close the browser
driver.quit()

In [47]:
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [48]:
from datetime import datetime

now = datetime.now()

dt_string = now.strftime("%d-%m-%Y_%H-%M-%S")

In [49]:
def check_exists_by_xpath(webdriver,xpath):
    try:
        webdriver.find_element(By.XPATH,xpath)
    except NoSuchElementException:
        return False
    return True

def check_exists_by_class(webdriver,classname):
    try:
        webdriver.find_element(By.CLASS_NAME,classname)
    except NoSuchElementException:
        return False
    return True

In [50]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.headless = True
driver = webdriver.Chrome(options=chrome_options)

In [51]:
check_xpath='/html/body/div[1]/div[7]/div[6]/div/div[2]/div/div[1]/div[2]/select[3]'

publisher_selector='#game_highlights > div.rightcol > div > div.glance_ctn_responsive_left > div:nth-child(4) > div.summary.column > a'
system_requirements_xpath='/html/body/div[1]/div[7]/div[6]/div[3]/div[2]/div[1]/div[5]/div[2]/div[8]/div[1]/div[1]'

os_system_requirements_xpath='/html/body/div[1]/div[7]/div[6]/div[3]/div[2]/div[1]/div[5]/div[2]/div[8]/div[1]/div[1]/div[1]'

In [52]:
list_game_detail=[]

In [53]:
miss=[]
for i in range(len(df)):
  game_detail={}
  name=[]
  try:
    driver.get(df.link[i])
    if check_exists_by_xpath(driver,check_xpath):
      option=driver.find_element(By.XPATH,'/html/body/div[1]/div[7]/div[6]/div/div[2]/div/div[1]/div[2]/select[3]')
      option.click()
      option=driver.find_element(By.XPATH,'/html/body/div[1]/div[7]/div[6]/div/div[2]/div/div[1]/div[2]/select[3]/option[103]')
      option.click()
      option=driver.find_element(By.XPATH,'/html/body/div[1]/div[7]/div[6]/div/div[2]/div/div[1]/div[3]/div/a[1]')
      option.click()

    name=df.name[i]
    genre=[element.text for element in driver.find_elements(By.CSS_SELECTOR, '#genresAndManufacturer > span')]
    feature_list=[element.text for element in driver.find_elements(By.CLASS_NAME, 'label')]

    if check_exists_by_class(driver,'all_languages'):
      all_languages=driver.find_element(By.CLASS_NAME,'all_languages')
      all_languages.click()
    supported_languages=[element.text for element in driver.find_elements(By.CLASS_NAME, 'ellipsis')]

    description=driver.find_element(By.CLASS_NAME, 'game_description_snippet').text

    release_date=driver.find_element(By.CLASS_NAME, 'date').text
    developer=driver.find_element(By.ID, 'developers_list').text
    publisher=driver.find_element(By.CSS_SELECTOR, publisher_selector).text

    num_curators=driver.find_element(By.CLASS_NAME, 'no_curators_followed').text


    wait = WebDriverWait(driver, 10)
    element = wait.until(EC.presence_of_element_located((By.CLASS_NAME,'user_reviews_filter_menu')))
    elm=driver.find_element(By.CLASS_NAME,'user_reviews_filter_menu')
    num_reviews=[]
    action=ActionChains(driver)


    num_reviews=['']
    while len(num_reviews[0])==0:
      action.move_to_element(elm)
      action.perform()
      num_reviews=[element.text for element in driver.find_elements(By.CSS_SELECTOR, '#reviews_filter_options > div:nth-child(1) > div.user_reviews_filter_menu_flyout > div')]


    requirements=[]
    if check_exists_by_xpath(driver,os_system_requirements_xpath):
      system_requirements=[element.text for element in driver.find_elements(By.XPATH, system_requirements_xpath)]
      l=len(system_requirements[0].split('\n'))
      cnt=1
      while cnt<=l:
        os_system_requirements_xpath=f'/html/body/div[1]/div[7]/div[6]/div[3]/div[2]/div[1]/div[5]/div[2]/div[8]/div[1]/div[1]/div[{cnt}]'
        os_system_requirements=driver.find_element(By.XPATH, os_system_requirements_xpath)
        action.click(os_system_requirements).perform()
        detail_requirements_xpath=f'/html/body/div[1]/div[7]/div[6]/div[3]/div[2]/div[1]/div[5]/div[2]/div[8]/div[1]/div[2]/div[{cnt}]/div[1]'
        detail_requirements=driver.find_element(By.XPATH, detail_requirements_xpath).text
        requirements.append(detail_requirements)
        cnt+=1
    else:
        single_os_requirements=driver.find_element(By.CLASS_NAME,'sysreq_contents').text
        requirements.append(single_os_requirements)

    game_detail={'name':name,
                  'genre':genre,
                  'description': description,
                  'feature_list':feature_list,
                  'supported_languages':supported_languages,
                  'release_date':release_date,
                  'developer':developer,
                  'publisher':publisher,
                  'num_curators':num_curators,
                  'num_reviews':num_reviews,
                  'requirements':requirements
                  }
    list_game_detail.append(game_detail)
  except:
    try:
      error=driver.find_element(By.ID,'error_box').text
      # print(error)
      game_detail={'name':df.name[i],
                  'genre':'0',
                  'description': '0',
                  'feature_list':'0',
                  'supported_languages':'0',
                  'release_date':'0',
                  'developer':'0',
                  'publisher':'0',
                  'num_curators':'0',
                  'num_reviews':'0',
                  'requirements':'0'
                  }
      list_game_detail.append(game_detail)

    except:
      miss.append(i)
      pass


An error was encountered while processing your request:

This item is currently unavailable in your region
An error was encountered while processing your request:

This item is currently unavailable in your region
An error was encountered while processing your request:

This item is currently unavailable in your region
An error was encountered while processing your request:

This item is currently unavailable in your region
An error was encountered while processing your request:

This item is currently unavailable in your region


In [54]:
list_game_detail_df=pd.DataFrame(list_game_detail)
list_game_detail_df

,name,genre,description,feature_list,supported_languages,release_date,developer,publisher,num_curators,num_reviews,requirements
0,Counter-Strike 2,"[Action, Free to Play]","For over two decades, Counter-Strike has offer...","[Cross-Platform Multiplayer, Steam Trading Car...","[English, Czech, Danish, Dutch, Finnish, Frenc...","21 Aug, 2012",Valve,Valve,"17,564 Curators have reviewed this product. Cl...","[All (7,686,243)\nPositive (6,766,697)\nNegati...",[MINIMUM:\nOS: Windows® 10\nProcessor: 4 hardw...
1,Dota 2,"[Action, Strategy, Free to Play]","Every day, millions of players worldwide enter...","[Steam Trading Cards, Steam Workshop, SteamVR ...","[English, Bulgarian, Czech, Danish, Dutch, Fin...","10 Jul, 2013",Valve,Valve,"5,123 Curators have reviewed this product. Cli...","[All (2,076,700)\nPositive (1,705,517)\nNegati...",[MINIMUM:\nOS: Windows 7 or newer\nProcessor: ...
2,PUBG: BATTLEGROUNDS,"[Adventure, RPG, Strategy]","Baldur’s Gate 3 is a story-rich, party-based R...","[Single-player, Online Co-op, LAN Co-op, Steam...","[English, French, German, Spanish - Spain, Pol...","3 Aug, 2023",Larian Studios,Larian Studios,558 Curators have reviewed this product. Click...,"[All (370,258)\nPositive (355,348)\nNegative (...",[MINIMUM:\nRequires a 64-bit processor and ope...
3,Apex Legends™,"[Action, Adventure, Free to Play]","Apex Legends is the award-winning, free-to-pla...","[Online PvP, Online Co-op, Steam Achievements,...","[English, French, Italian, German, Spanish - S...","5 Nov, 2020",Respawn Entertainment,Electronic Arts,935 Curators have reviewed this product. Click...,"[All (732,373)\nPositive (584,377)\nNegative (...",[MINIMUM:\nRequires a 64-bit processor and ope...
4,Team Fortress 2,"[Action, Free to Play]",Nine distinct classes provide a broad range of...,"[Cross-Platform Multiplayer, Steam Achievement...","[English, Danish, Dutch, Finnish, French, Germ...","10 Oct, 2007",Valve,Valve,"9,467 Curators have reviewed this product. Cli...","[All (997,008)\nPositive (936,120)\nNegative (...",[MINIMUM:\nOS: Windows® 7 (32/64-bit)/Vista/XP...
...,...,...,...,...,...,...,...,...,...,...,...
89,Albion Online,"[Massively Multiplayer, RPG, Free to Play]",Albion Online is a fantasy sandbox MMORPG feat...,"[MMO, Online PvP, Cross-Platform Multiplayer, ...","[English, French, German, Spanish - Spain, Por...","17 Jul, 2017",Sandbox Interactive GmbH,Sandbox Interactive GmbH,187 Curators have reviewed this product. Click...,"[All (64,748)\nPositive (51,431)\nNegative (13...",[MINIMUM:\nRequires a 64-bit processor and ope...
90,MY HERO ULTRA RUMBLE,"[Action, Casual, Free to Play]","Pick your favorite character, team up and figh...","[Single-player, Online PvP, Online Co-op, Stea...","[English, Japanese, French, Italian, German, S...","29 Sep, 2023",Byking Inc.,Bandai Namco Entertainment,27 Curators have reviewed this product. Click ...,"[All (10,262)\nPositive (6,654)\nNegative (3,6...",[MINIMUM:\nRequires a 64-bit processor and ope...
91,Age of Empires IV: Anniversary Edition,[Strategy],Celebrating its first year of delighting milli...,"[Single-player, Online PvP, Online Co-op, Cros...","[English, French, Italian, German, Spanish - S...","28 Oct, 2021","Relic Entertainment, Forgotten Empires, Climax...",Xbox Game Studios,221 Curators have reviewed this product. Click...,"[All (56,666)\nPositive (49,011)\nNegative (7,...",[MINIMUM:\nRequires a 64-bit processor and ope...
92,MARVEL SNAP,"[Casual, Strategy, Free to Play]",MARVEL SNAP is a fast-paced Collectible Card G...,"[Online PvP, Cross-Platform Multiplayer, In-Ap...","[English, French, Italian, German, Spanish - S...","23 Aug, 2023","Second Dinner Studios, Inc.",Nuverse,101 Curators have reviewed this product. Click...,"[All (18,959)\nPositive (15,684)\nNegative (3,...",[MINIMUM:\nOS: Windows 7 (SP1+)\nProcessor: In...


In [56]:
merge=df.merge(list_game_detail_df, left_on='name', right_on='name')
merge.to_json(f'./{dt_string}.json',orient="index",indent=2)
